## Chainlitの練習

In [ ]:
#オウム返し
import chainlit as cl

@cl.on_message
async def main(message: cl.Message):
    # Your custom logic goes here...

    # Send a response back to the user
    await cl.Message(
        content=f"Received: {message.content}",
    ).send()

In [ ]:
import chainlit as cl
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

#モデルを設定
chat = ChatOpenAI(
    model="gpt-4o",
)

#バッファーを初期化
memory = ConversationBufferMemory(
    return_messages=True,
)


@cl.on_message
async def main(message: cl.Message):
    # Your custom logic goes here...
    chat = ChatOpenAI(
        #ストリーミングモード指定
        streaming=True,
        callbacks=[
            StreamingStdOutCallbackHandler()
        ]
    )    
    #リクエスト
    result = chat([
        HumanMessage(content=message.content)
    ])
    #
    await cl.Message(content=f"GPT: {result.content}").send()    
    # Send a response back to the user
    # await cl.Message(
    #     content=f"GPT: {message.content}",
    # ).send()

# OpenAIのAPIを使う場合

In [ ]:
from openai import AsyncOpenAI
import chainlit as cl
client = AsyncOpenAI()

# Instrument the OpenAI client
cl.instrument_openai()

settings = {
    "model": "gpt-3.5-turbo",
    "temperature": 1,
    # ... more settings
}

@cl.on_message
async def on_message(message: cl.Message):
    response = await client.chat.completions.create(
        messages=[
            {
                "content": "You are a helpful bot, you always reply in Japanes",
                "role": "system"
            },
            {
                "content": message.content,
                "role": "user"
            }
        ],
        **settings
    )
    await cl.Message(content=response.choices[0].message.content).send()


## LangChainを使ったやり方

In [ ]:
#逐次表示されるチャット
#from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import Runnable
from langchain.schema.runnable.config import RunnableConfig

import chainlit as cl

@cl.on_chat_start
async def on_chat_start():
    model = ChatOpenAI(streaming=True)
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You're a very knowledgeable historian who provides accurate and eloquent answers to historical questions.",
            ),
            ("human", "{question}"),
        ]
    )
    runnable = prompt | model | StrOutputParser()
    cl.user_session.set("runnable", runnable)


@cl.on_message
async def on_message(message: cl.Message):
    runnable = cl.user_session.get("runnable")  # type: Runnable

    msg = cl.Message(content="")

    async for chunk in runnable.astream(
        {"question": message.content},
        config=RunnableConfig(callbacks=[cl.LangchainCallbackHandler()]),
    ):
        await msg.stream_token(chunk)

    await msg.send()


In [ ]:
#[Deprecated] With Legacy Chain Interface
#逐次表示されないバージョン
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.chains import LLMChain

import chainlit as cl


@cl.on_chat_start
async def on_chat_start():
    model = ChatOpenAI(streaming=True)
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You're a very knowledgeable historian who provides accurate and eloquent answers to historical questions.",
            ),
            ("human", "{question}"),
        ]
    )
    chain = LLMChain(llm=model, prompt=prompt, output_parser=StrOutputParser())

    cl.user_session.set("chain", chain)


@cl.on_message
async def on_message(message: cl.Message):
    chain = cl.user_session.get("chain")  # type: LLMChain

    res = await chain.arun(
        question=message.content, callbacks=[cl.LangchainCallbackHandler()]
    )

    await cl.Message(content=res).send()


## 自主研究

In [7]:
#ChatGPT 4oの場合
#逐次表示されるチャット
#from langchain_openai import ChatOpenAI
#from langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import Runnable
from langchain.schema.runnable.config import RunnableConfig

import chainlit as cl

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="私はGPT-4oです、何でも質問してください。").send()
    model = ChatOpenAI(
        model="gpt-4o",
        streaming=True
    )
    #
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "あなたは優秀なアシスタントです。",
            ),
            ("human", "{question}"),
        ]
    )
    #
    runnable = prompt | model | StrOutputParser()
    #
    cl.user_session.set("runnable", runnable)


@cl.on_message
async def on_message(message: cl.Message):
    runnable = cl.user_session.get("runnable")  # type: Runnable

    msg = cl.Message(content="")

    async for chunk in runnable.astream(
        {"question": message.content},
        config=RunnableConfig(callbacks=[cl.LangchainCallbackHandler()]),
    ):
        await msg.stream_token(chunk)

    await msg.send()

In [4]:
#Geminiの場合
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import Runnable
from langchain.schema.runnable.config import RunnableConfig

import chainlit as cl


@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="私はGemini-1.5-Flashです、何でも質問してください。").send()
    model = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash-latest",
        streaming=True
    )
    #
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "あなたは優秀なアシスタントです。",
            ),
            ("human", "{question}"),
        ]
    )
    #
    runnable = prompt | model | StrOutputParser()
    #
    cl.user_session.set("runnable", runnable)


@cl.on_message
async def on_message(message: cl.Message):
    runnable = cl.user_session.get("runnable")  # type: Runnable

    msg = cl.Message(content="")

    async for chunk in runnable.astream(
        {"question": message.content},
        config=RunnableConfig(callbacks=[cl.LangchainCallbackHandler()]),
    ):
        await msg.stream_token(chunk)

    await msg.send()

In [6]:
#Geminiの場合
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import Runnable
from langchain.schema.runnable.config import RunnableConfig

import chainlit as cl


@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="私はgemini-1.5-proです、何でも質問してください。").send()
    model = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro-latest",
        streaming=True
    )
    #
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "あなたは優秀なアシスタントです。",
            ),
            ("human", "{question}"),
        ]
    )
    #
    runnable = prompt | model | StrOutputParser()
    #
    cl.user_session.set("runnable", runnable)


@cl.on_message
async def on_message(message: cl.Message):
    runnable = cl.user_session.get("runnable")  # type: Runnable

    msg = cl.Message(content="")

    async for chunk in runnable.astream(
        {"question": message.content},
        config=RunnableConfig(callbacks=[cl.LangchainCallbackHandler()]),
    ):
        await msg.stream_token(chunk)

    await msg.send()

## 公式ドキュメントより

In [1]:
import os

import google.generativeai as genai

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
# See https://ai.google.dev/api/python/google/generativeai/GenerativeModel
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
]

model = genai.GenerativeModel(
    #model_name="gemini-pro", 
  #model_name="gemini-1.5-pro-latest", 
    model_name="gemini-1.5-flash-latest",
  safety_settings=safety_settings,
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
  ]
)

response = chat_session.send_message("マグロはいつ休む？")

print(response.text)
print(chat_session.history)

 マグロは常に活動している魚です。マグロは水温を調節するために、絶えず動き続けています。マグロは、泳いでいるときに筋肉が熱を発生させ、その熱を体内に保持します。そのため、マグロは長時間休むことはできません。マグロは、短い時間だけ休むために、水中に潜ったり、水面近くで漂ったりします。 

マグロは、獲物を追いかけるためにも、常に活動する必要があります。マグロは、速い速度で泳ぎ、遠くまで移動することができます。そのため、マグロは、休む時間を最小限に抑えなければなりません。 

マグロは、水温の変化や餌の量によって、活動のレベルが異なります。水温が低いときや餌が少ないときは、マグロは活動レベルが低下します。しかし、マグロは、常に活動している魚であり、休む時間は非常に短いです。

[parts {
  text: "マグロはいつ休む？"
}
role: "user"
, parts {
  text: " マグロは常に活動している魚です。マグロは水温を調節するために、絶えず動き続けています。マグロは、泳いでいるときに筋肉が熱を発生させ、その熱を体内に保持します。そのため、マグロは長時間休むことはできません。マグロは、短い時間だけ休むために、水中に潜ったり、水面近くで漂ったりします。 \n\nマグロは、獲物を追いかけるためにも、常に活動する必要があります。マグロは、速い速度で泳ぎ、遠くまで移動することができます。そのため、マグロは、休む時間を最小限に抑えなければなりません。 \n\nマグロは、水温の変化や餌の量によって、活動のレベルが異なります。水温が低いときや餌が少ないときは、マグロは活動レベルが低下します。しかし、マグロは、常に活動している魚であり、休む時間は非常に短いです。\n"
}
role: "model"
]


In [ ]:
import os

os.environ